In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json

In [4]:
def get_target_zips(city, state, zip = None):
    zips = pd.read_csv("zip_code_database.csv")
    if zip == None:
        target_zips = zips[(zips["primary_city"] == city) & (zips["state"] == state)]["zip"].tolist()
    else:
        target_zips = [zip]
    return target_zips


def get_stingray_rgn_id(zip):
    query_location_api = f"https://www.redfin.com/stingray/do/query-location?location={zip}&v=2"
    response = requests.get(query_location_api, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}) 
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    data = json.loads(prefix_removed)
    try:
        region_id = data["payload"]["exactMatch"].get("id").split("_",1)[1]
        return region_id
    except:
        print(f"No Exact match found for zip: {zip}")
        return None


def build_stingray_gis_params(params):
        return "&".join(f"{key}={value}" for key, value in params.items() if params.get(key) != None)


def call_stingray_gis(params_url):
    api_url = "https://www.redfin.com/stingray/api/gis"
    url = f"{api_url}?{params_url}"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    soup = BeautifulSoup(response.text, 'html.parser').text
    prefix_removed = soup.split('&&', 1)[1]
    # print(url)
    data = json.loads(prefix_removed)

    return data


def parse_stingray_gis(data):
    homes = data.get('payload', {}).get('homes', [])
    # print(homes)
    parsed_homes = []
    
    for home in homes:
        home_info = {
            "MLS ID": home.get('mlsId', {}).get('value'),
            "Status": home.get('mlsStatus'),
            "Price": home.get('price', {}).get('value'),
            "HOA Fee": home.get('hoa', {}).get('value'),
            "Square Feet": home.get('sqFt', {}).get('value'),
            "Price per Square Foot": home.get('pricePerSqFt', {}).get('value'),
            "Lot Size": home.get('lotSize', {}).get('value'),
            "Bedrooms": home.get('beds'),
            "Bathrooms": home.get('baths'),
            "Location": home.get('location', {}).get('value'),
            "Stories": home.get('stories'),
            "Address": home.get('streetLine', {}).get('value'),
            "City": home.get('city'),
            "State": home.get('state'),
            "ZIP Code": home.get('postalCode', {}).get('value'),
            "Year Built": home.get('yearBuilt', {}).get('value'),
            "URL": home.get('url'),
        }
        parsed_homes.append(home_info)
    
    return parsed_homes

In [5]:
# GIS Search API

import requests

zip = None
City = "Chicago"
State = "IL"

target_zips = get_target_zips(City, State)

print(target_zips)

data = []

for zip in target_zips:
    params = {
    #??Active Listings
    "al": 1,
    #Include Nearby Homes
    "include_nearby_homes": False,
    # Market. ie Seattle
    "market": None,
    # Number of homes to retrieve
    "num_homes": 350,
    #How to Sort the homes
    "ord": "days-on-redfin-asc",
    "page_number": 1,
    "poly": None,
    #Listing Types
    "sf": "1,2,3,4,5,6,7",
    "start": None,
    "status": 9,
    # User input property types (currently only single family, townhomes, multifamily : 134)
    "uipt": "1,3,4",
    # ??API Version?
    "v": 8,
    "zoomLevel": None,
    #Type of Region analyzed
    "region_type" : 2,
    "region_id" : get_stingray_rgn_id(zip)
    }

    if params.get("region_id") == None:
        continue
    else:
        url_param = build_stingray_gis_params(params)
        json_data = call_stingray_gis(url_param)
        list_data = parse_stingray_gis(json_data)
        
        data.extend(list_data)

df= pd.DataFrame(data)

[60290, 60601, 60602, 60603, 60604, 60605, 60606, 60607, 60608, 60609, 60610, 60611, 60612, 60613, 60614, 60615, 60616, 60617, 60618, 60619, 60620, 60621, 60622, 60623, 60624, 60625, 60626, 60628, 60629, 60630, 60631, 60632, 60633, 60634, 60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643, 60644, 60645, 60646, 60647, 60649, 60651, 60652, 60653, 60654, 60655, 60656, 60657, 60659, 60660, 60661, 60663, 60664, 60666, 60668, 60669, 60670, 60673, 60674, 60675, 60677, 60678, 60679, 60680, 60681, 60682, 60684, 60685, 60686, 60687, 60688, 60689, 60690, 60691, 60693, 60694, 60695, 60696, 60697, 60699, 60701]
No Exact match found for zip: 60290


In [10]:
df.drop_duplicates(subset=["MLS ID"], inplace=True)

display(df)


,MLS ID,Status,Price,HOA Fee,Square Feet,Price per Square Foot,Lot Size,Bedrooms,Bathrooms,Location,Stories,Address,City,State,ZIP Code,Year Built,URL
0,12075800,Active,539900.0,NaN,NaN,NaN,3780.0,5.0,2.0,CHI - Lower West Side,NaN,1742 W Cullerton St,Chicago,IL,60608,1875.0,/IL/Chicago/1742-W-Cullerton-St-60608/home/140...
1,12068987,Active,700000.0,NaN,NaN,NaN,2613.0,7.0,4.0,CHI - Bridgeport,NaN,2812 S Throop St,Chicago,IL,60608,1860.0,/IL/Chicago/2812-S-Throop-St-60608/home/14079827
2,12002683,Active,679900.0,363.0,1907.0,357.0,NaN,3.0,2.5,University Village,3.0,841 W Village Ct,Chicago,IL,60608,2002.0,/IL/Chicago/841-W-Village-Ct-60608/home/22914928
3,12073887,Active,589999.0,NaN,NaN,NaN,3696.0,8.0,3.0,CHI - South Lawndale,NaN,2629 W Luther St,Chicago,IL,60608,NaN,/IL/Chicago/2629-W-Luther-St-60608/home/191041851
4,12074610,Active,370000.0,NaN,1980.0,187.0,3267.0,5.0,3.0,CHI - McKinley Park,2.0,3422 S Hermitage Ave,Chicago,IL,60608,1900.0,/IL/Chicago/3422-S-Hermitage-Ave-60608/home/14...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806,11912719,Active,1999000.0,NaN,NaN,NaN,NaN,9.0,8.0,CHI - Near West Side,3.0,1502 W Jackson Blvd,Chicago,IL,60607,1885.0,/IL/Chicago/1502-W-Jackson-Blvd-60607/home/140...
2808,11910563,Active,765000.0,265.0,NaN,NaN,NaN,3.0,2.5,Dearborn Park II,4.0,1431 S Clark St,Chicago,IL,60605,1993.0,/IL/Chicago/1431-S-Clark-St-60605/home/14084094
2821,12066686,Active,1325000.0,814.0,3820.0,347.0,NaN,3.0,3.5,CHI - Near South Side,3.0,1439 S Prairie Ave Unit B,Chicago,IL,60605,2003.0,/IL/Chicago/1439-S-Prairie-Ave-60605/unit-B/ho...
2840,12026868,Active,999985.0,518.0,NaN,NaN,NaN,3.0,2.5,CHI - Near South Side,1.0,1415 S Indiana Ave Unit A,Chicago,IL,60605,2005.0,/IL/Chicago/1415-S-Indiana-Ave-60605/unit-A/ho...
